# Project: Which customer will respond to direct mail? 

 **by Group 7**


# Problem Context
Springleaf puts the humanity back into lending by offering their customers personal and auto loans that help them take control of their lives and their finances. Direct mail is one important way Springleaf's team can connect with customers whom may be in need of a loan.



Direct offers provide huge value to customers who need them, and are a fundamental part of Springleaf's marketing strategy. In order to improve their targeted efforts, Springleaf must be sure they are focusing on the customers who are likely to respond and be good candidates for their services.

Using a large set of anonymized features, Springleaf is asking you to predict which customers will respond to a direct mail offer. You are challenged to engineer construct new features and employ feature-selection methods to approach this dauntingly wide dataset.

# Importing Libraries

In [52]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Mounting drive to fetch data

In [53]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [54]:
df_train = pd.read_csv('/content/drive/MyDrive/Mail_Project_Data/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Mail_Project_Data/test.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,9,10,11,12,43,157,196,214,225,228,229,231,235,238) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,9,10,11,12,43,196,214,225,228,229,231,235,238) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [55]:
# Looking at training data set
df_train 

,ID,VAR_0001,VAR_0002,VAR_0003,VAR_0004,VAR_0005,VAR_0006,VAR_0007,VAR_0008,VAR_0009,VAR_0010,VAR_0011,VAR_0012,VAR_0013,VAR_0014,VAR_0015,VAR_0016,VAR_0017,VAR_0018,VAR_0019,VAR_0020,VAR_0021,VAR_0022,VAR_0023,VAR_0024,VAR_0025,VAR_0026,VAR_0027,VAR_0028,VAR_0029,VAR_0030,VAR_0031,VAR_0032,VAR_0033,VAR_0034,VAR_0035,VAR_0036,VAR_0037,VAR_0038,VAR_0039,...,VAR_1896,VAR_1897,VAR_1898,VAR_1899,VAR_1900,VAR_1901,VAR_1902,VAR_1903,VAR_1904,VAR_1905,VAR_1906,VAR_1907,VAR_1908,VAR_1909,VAR_1910,VAR_1911,VAR_1912,VAR_1913,VAR_1914,VAR_1915,VAR_1916,VAR_1917,VAR_1918,VAR_1919,VAR_1920,VAR_1921,VAR_1922,VAR_1923,VAR_1924,VAR_1925,VAR_1926,VAR_1927,VAR_1928,VAR_1929,VAR_1930,VAR_1931,VAR_1932,VAR_1933,VAR_1934,target
0,2,H,224,0,4300,C,0.0,0.0,False,False,False,False,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,997.0,997.0,997.0,147.0,198.0,173.0,0.0,98.0,98.0,98.0,98.0,98.0,98.0,98.0,998.0,999999998.0,999999998.0,999999998.0,999999998.0,998.0,998.0,9998.0,9998.0,0.0,98.0,999999998.0,999999998.0,9998.0,0.0,98.0,98.0,998.0,999999998.0,998.0,998.0,9998.0,9998.0,IAPS,0.0
1,4,H,7,53,4448,B,1.0,0.0,False,False,False,False,False,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,49.0,49.0,3.0,21.0,12.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,6546.0,6546.0,6125.0,76.0,1.0,1.0,9996.0,111.0,0.0,98.0,999999998.0,999999998.0,9998.0,0.0,98.0,98.0,998.0,999999998.0,998.0,998.0,9998.0,9998.0,IAPS,0.0
2,5,H,116,3,3464,C,0.0,0.0,False,False,False,False,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,997.0,997.0,45.0,46.0,46.0,1.0,0.0,0.0,3.0,0.0,97.0,0.0,97.0,997.0,999999997.0,999999997.0,999999997.0,999999997.0,1.0,997.0,9996.0,113.0,0.0,98.0,999999998.0,999999998.0,9998.0,0.0,98.0,98.0,998.0,999999998.0,998.0,998.0,9998.0,9998.0,IAPS,0.0
3,7,H,240,300,3200,C,0.0,0.0,False,False,False,False,False,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,120.0,30.0,996.0,996.0,15.0,188.0,106.0,0.0,98.0,98.0,98.0,98.0,98.0,98.0,98.0,998.0,999999998.0,999999998.0,999999998.0,999999998.0,998.0,998.0,9998.0,9998.0,0.0,98.0,999999998.0,999999998.0,9998.0,0.0,98.0,98.0,998.0,999999998.0,998.0,998.0,9998.0,9998.0,RCC,0.0
4,8,R,72,261,2000,N,0.0,0.0,False,False,False,False,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,30.0,1.0,17.0,17.0,76.0,443.0,230.0,0.0,98.0,98.0,98.0,98.0,98.0,98.0,98.0,998.0,999999998.0,999999998.0,999999998.0,999999998.0,998.0,998.0,9998.0,9998.0,0.0,98.0,999999998.0,999999998.0,9998.0,0.0,98.0,98.0,998.0,999999998.0,998.0,998.0,9998.0,9998.0,BRANCH,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80249,160768,R,12,5,1660,N,0.0,0.0,False,False,False,False,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,31.0,31.0,4.0,36.0,17.0,0.0,98.0,98.0,98.0,98.0,98.0,98.0,98.0,998.0,999999998.0,999999998.0,999999998.0,999999998.0,998.0,998.0,9998.0,9998.0,0.0,98.0,999999998.0,999999998.0,9998.0,0.0,98.0,98.0,998.0,999999998.0,998.0,998.0,9998.0,9998.0,BRANCH,0.0
80250,160773,R,2,0,2100,C,0.0,0.0,False,False,False,False,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,998.0,998.0,998.0,998.0,9998.0,9998.0,9998.0,0.0,98.0,98.0,98.0,98.0,98.0,98.0,98.0,998.0,999999998.0,999999998.0,999999998.0,999999998.0,998.0,998.0,9998.0,9998.0,0.0,98.0,999999998.0,999999998.0,9998.0,0.0,98.0,98.0,998.0,9

In [56]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80254 entries, 0 to 80253
Columns: 1934 entries, ID to target
dtypes: float64(1012), int64(871), object(51)
memory usage: 1.2+ GB


# Data Cleaning

In [57]:
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
VAR_0840,80254,1.000000
VAR_0213,80254,1.000000
VAR_0207,80254,1.000000
VAR_0214,80246,0.999900
VAR_0157,79725,0.993408
VAR_0158,79095,0.985558
VAR_0205,79048,0.984973
VAR_0206,78992,0.984275
VAR_0167,78872,0.982780
VAR_0177,78420,0.977148


In [58]:
missing_data[missing_data["Total"] > 1] 

,Total,Percent
VAR_0840,80254,1.000000
VAR_0213,80254,1.000000
VAR_0207,80254,1.000000
VAR_0214,80246,0.999900
VAR_0157,79725,0.993408
...,...,...
VAR_0088,33,0.000411
VAR_0087,33,0.000411
VAR_0086,33,0.000411
VAR_0085,33,0.000411


# Removal of missing data from the set

In [59]:
df_train = df_train.drop((missing_data[missing_data['Total'] > 1]).index,1)
df_test = df_test.drop((missing_data[missing_data['Total'] > 1]).index,1)

In [60]:
df_train
#below we can see that missing data is removed and dataset is updated
#columns have been reduced from 1934 to 1409

,ID,VAR_0001,VAR_0002,VAR_0003,VAR_0004,VAR_0005,VAR_0532,VAR_0533,VAR_0534,VAR_0535,VAR_0536,VAR_0537,VAR_0538,VAR_0539,VAR_0540,VAR_0541,VAR_0542,VAR_0543,VAR_0544,VAR_0545,VAR_0546,VAR_0547,VAR_0548,VAR_0549,VAR_0550,VAR_0551,VAR_0552,VAR_0553,VAR_0554,VAR_0555,VAR_0556,VAR_0557,VAR_0558,VAR_0559,VAR_0560,VAR_0561,VAR_0562,VAR_0563,VAR_0564,VAR_0565,...,VAR_1896,VAR_1897,VAR_1898,VAR_1899,VAR_1900,VAR_1901,VAR_1902,VAR_1903,VAR_1904,VAR_1905,VAR_1906,VAR_1907,VAR_1908,VAR_1909,VAR_1910,VAR_1911,VAR_1912,VAR_1913,VAR_1914,VAR_1915,VAR_1916,VAR_1917,VAR_1918,VAR_1919,VAR_1920,VAR_1921,VAR_1922,VAR_1923,VAR_1924,VAR_1925,VAR_1926,VAR_1927,VAR_1928,VAR_1929,VAR_1930,VAR_1931,VAR_1932,VAR_1933,VAR_1934,target
0,2,H,224,0,4300,C,5,0,0,5,3,0,0,0,84,49463,0,116783,3230,1,998,998,998,998,66,9996,96,96,9996,9996,9996,9996,9996,96,96,0,0,0,0,0,...,1.0,997.0,997.0,997.0,147.0,198.0,173.0,0.0,98.0,98.0,98.0,98.0,98.0,98.0,98.0,998.0,999999998.0,999999998.0,999999998.0,999999998.0,998.0,998.0,9998.0,9998.0,0.0,98.0,999999998.0,999999998.0,9998.0,0.0,98.0,98.0,998.0,999999998.0,998.0,998.0,9998.0,9998.0,IAPS,0.0
1,4,H,7,53,4448,B,0,0,0,3,12,0,0,0,63,303472,0,346196,899,1,998,998,998,998,99,9996,96,96,9996,9996,9996,9996,9996,96,96,9,0,0,0,0,...,1.0,1.0,49.0,49.0,3.0,21.0,12.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,6546.0,6546.0,6125.0,76.0,1.0,1.0,9996.0,111.0,0.0,98.0,999999998.0,999999998.0,9998.0,0.0,98.0,98.0,998.0,999999998.0,998.0,998.0,9998.0,9998.0,IAPS,0.0
2,5,H,116,3,3464,C,0,0,0,1,2,0,0,0,48,94990,0,122601,0,1,998,998,998,998,87,9996,96,96,21,64,9996,9996,9996,96,96,25,0,0,0,0,...,1.0,1.0,997.0,997.0,45.0,46.0,46.0,1.0,0.0,0.0,3.0,0.0,97.0,0.0,97.0,997.0,999999997.0,999999997.0,999999997.0,999999997.0,1.0,997.0,9996.0,113.0,0.0,98.0,999999998.0,999999998.0,9998.0,0.0,98.0,98.0,998.0,999999998.0,998.0,998.0,9998.0,9998.0,IAPS,0.0
3,7,H,240,300,3200,C,0,0,0,1,6,2,0,0,65,20593,0,59490,286,1,998,998,400,400,77,9996,1,1,3,78,75,74,71,16,16,19,0,0,0,0,...,120.0,30.0,996.0,996.0,15.0,188.0,106.0,0.0,98.0,98.0,98.0,98.0,98.0,98.0,98.0,998.0,999999998.0,999999998.0,999999998.0,999999998.0,998.0,998.0,9998.0,9998.0,0.0,98.0,999999998.0,999999998.0,9998.0,0.0,98.0,98.0,998.0,999999998.0,998.0,998.0,9998.0,9998.0,RCC,0.0
4,8,R,72,261,2000,N,0,0,0,1,8,0,0,0,84,10071,0,35708,947,1,1,30,998,998,57,9996,96,96,41,35,9996,9996,9996,96,96,0,0,0,0,0,...,30.0,1.0,17.0,17.0,76.0,443.0,230.0,0.0,98.0,98.0,98.0,98.0,98.0,98.0,98.0,998.0,999999998.0,999999998.0,999999998.0,999999998.0,998.0,998.0,9998.0,9998.0,0.0,98.0,999999998.0,999999998.0,9998.0,0.0,98.0,98.0,998.0,999999998.0,998.0,998.0,9998.0,9998.0,BRANCH,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80249,160768,R,12,5,1660,N,0,0,1,0,4,2,0,0,54,1678,0,6000,500,1,998,998,998,998,45,9996,96,96,9996,9996,9996,9996,9996,96,96,0,0,0,0,0,...,1.0,1.0,31.0,31.0,4.0,36.0,17.0,0.0,98.0,98.0,98.0,98.0,98.0,98.0,98.0,998.0,999999998.0,999999998.0,999999998.0,999999998.0,998.0,998.0,9998.0,9998.0,0.0,98.0,999999998.0,999999998.0,9998.0,0.0,98.0,98.0,998.0,999999998.0,998.0,998.0,9998.0,9998.0,BRANCH,0.0
80250,160773,R,2,0,2100,C,0,0,0,0,1,1,0,1,4,710,7227,1119,0,1,998,998,998,998,63,9996,1,1,4,62,56,9990,52,14,5,6,0,0,0,0,...,998.0,998.0,998.0,998.0,9998.0,9998.0,9998.0,0.0,98.0,98.0,98.0,98.0,98.0,98.0,98.0,998.0,999999998.0,999999998.0,999999998.0,999999998.0,998.0,998.0,9998.0,9998.0,0.0,98.0,999999998.0,999999998.0,9998.0,0.0,98.0,98.0,998.0,999999998.0,998.0,998.0,9998.0,9998.0,IAPS,0.0
80251,160774,H,92,142,1693,B,1,0,0,1,1,1,0,0,74,56115,0,74524,993,1,998,998,998,998,77,9996,1,1,2,79,73,71,9996,19,8,20,0,0,0,0,...,60.0,1.0,0.0,0.0,26.0,132.0,73.0,0.0,98.0,98.0,98.0,98.0,98.0,98.0,98.0,998.0,999999998.0,999999998.0,9999999

In [61]:
df_train["target"].unique()

array([ 0.,  1., nan])

In [62]:
df_train.select_dtypes(include=['O'])
#Displaying categorical columns with fixed numner of values

,VAR_0001,VAR_0005,VAR_1934
0,H,C,IAPS
1,H,B,IAPS
2,H,C,IAPS
3,H,C,RCC
4,R,N,BRANCH
...,...,...,...
80249,R,N,BRANCH
80250,R,C,IAPS
80251,H,B,BRANCH
80252,H,B,BRANCH


In [63]:
df_train["VAR_0001"].unique()

array(['H', 'R', 'Q'], dtype=object)

In [64]:
#Conversion of categorical values to numerical
df_train['0001H'] = df_train['VAR_0001'].map({'H': 1, 'R': 0, 'Q': 0})
df_train['0001R'] = df_train['VAR_0001'].map({'H': 0, 'R': 1, 'Q': 0})
df_train['0001Q'] = df_train['VAR_0001'].map({'H': 0, 'R': 0, 'Q': 1})
df_train.pop("VAR_0001")

df_test['0001H'] = df_test['VAR_0001'].map({'H': 1, 'R': 0, 'Q': 0})
df_test['0001R'] = df_test['VAR_0001'].map({'H': 0, 'R': 1, 'Q': 0})
df_test['0001Q'] = df_test['VAR_0001'].map({'H': 0, 'R': 0, 'Q': 1})
df_test.pop("VAR_0001")


0       R
1       R
2       R
3       R
4       H
       ..
1888    R
1889    H
1890    H
1891    Q
1892    H
Name: VAR_0001, Length: 1893, dtype: object

In [65]:
df_train["VAR_0005"].unique()

array(['C', 'B', 'N', 'S'], dtype=object)

In [66]:
df_train['0005C'] = df_train['VAR_0005'].map({'C': 1, 'B': 0, 'N': 0, 'S': 0})
df_train['0005B'] = df_train['VAR_0005'].map({'C': 1, 'B': 0, 'N': 0, 'S': 0})
df_train['0005N'] = df_train['VAR_0005'].map({'C': 1, 'B': 0, 'N': 0, 'S': 0})
df_train['0005S'] = df_train['VAR_0005'].map({'C': 1, 'B': 0, 'N': 0, 'S': 0})
df_train.pop("VAR_0005")

df_test['0005C'] = df_test['VAR_0005'].map({'C': 1, 'B': 0, 'N': 0, 'S': 0})
df_test['0005B'] = df_test['VAR_0005'].map({'C': 1, 'B': 0, 'N': 0, 'S': 0})
df_test['0005N'] = df_test['VAR_0005'].map({'C': 1, 'B': 0, 'N': 0, 'S': 0})
df_test['0005S'] = df_test['VAR_0005'].map({'C': 1, 'B': 0, 'N': 0, 'S': 0})
df_test.pop("VAR_0005")

0       B
1       C
2       C
3       B
4       C
       ..
1888    B
1889    B
1890    C
1891    N
1892    N
Name: VAR_0005, Length: 1893, dtype: object

In [67]:
df_train["VAR_1934"].unique()

array(['IAPS', 'RCC', 'BRANCH', 'MOBILE', 'CSC', nan], dtype=object)

In [68]:
#Converting categorical to numeric
df_train['1934_IAPS'] = df_train['VAR_1934'].map({'IAPS': 1, 'RCC': 0, 'BRANCH': 0, 'MOBILE': 0, 'CSC': 0})
df_train['1934_RCC'] = df_train['VAR_1934'].map({'IAPS': 0, 'RCC': 1, 'BRANCH': 0, 'MOBILE': 0, 'CSC': 0})
df_train['1934_BRANCH'] = df_train['VAR_1934'].map({'IAPS': 0, 'RCC': 0, 'BRANCH': 1, 'MOBILE': 0, 'CSC': 0})
df_train['1934_MOBILE'] = df_train['VAR_1934'].map({'IAPS': 0, 'RCC': 0, 'BRANCH': 0, 'MOBILE': 1, 'CSC': 0})
df_train['1934_CSC'] = df_train['VAR_1934'].map({'IAPS': 0, 'RCC': 0, 'BRANCH': 0, 'MOBILE': 0, 'CSC': 1})
df_train.pop("VAR_1934")

df_test['1934_IAPS'] = df_test['VAR_1934'].map({'IAPS': 1, 'RCC': 0, 'BRANCH': 0, 'MOBILE': 0, 'CSC': 0})
df_test['1934_RCC'] = df_test['VAR_1934'].map({'IAPS': 0, 'RCC': 1, 'BRANCH': 0, 'MOBILE': 0, 'CSC': 0})
df_test['1934_BRANCH'] = df_test['VAR_1934'].map({'IAPS': 0, 'RCC': 0, 'BRANCH': 1, 'MOBILE': 0, 'CSC': 0})
df_test['1934_MOBILE'] = df_test['VAR_1934'].map({'IAPS': 0, 'RCC': 0, 'BRANCH': 0, 'MOBILE': 1, 'CSC': 0})
df_test['1934_CSC'] = df_test['VAR_1934'].map({'IAPS': 0, 'RCC': 0, 'BRANCH': 0, 'MOBILE': 0, 'CSC': 1})
df_test.pop("VAR_1934")

0         IAPS
1         IAPS
2         IAPS
3         IAPS
4         IAPS
         ...  
1888    BRANCH
1889    BRANCH
1890      IAPS
1891    BRANCH
1892       NaN
Name: VAR_1934, Length: 1893, dtype: object

In [69]:
sum(df_train.select_dtypes(include=['O']))

0

# Replenishing missing data

In [70]:
df_train = df_train.fillna(-1)
df_test = df_test.fillna(-1)

In [71]:
df_train['target1'] = df_train["target"]
df_train.pop('target')
df_train

,ID,VAR_0002,VAR_0003,VAR_0004,VAR_0532,VAR_0533,VAR_0534,VAR_0535,VAR_0536,VAR_0537,VAR_0538,VAR_0539,VAR_0540,VAR_0541,VAR_0542,VAR_0543,VAR_0544,VAR_0545,VAR_0546,VAR_0547,VAR_0548,VAR_0549,VAR_0550,VAR_0551,VAR_0552,VAR_0553,VAR_0554,VAR_0555,VAR_0556,VAR_0557,VAR_0558,VAR_0559,VAR_0560,VAR_0561,VAR_0562,VAR_0563,VAR_0564,VAR_0565,VAR_0566,VAR_0567,...,VAR_1907,VAR_1908,VAR_1909,VAR_1910,VAR_1911,VAR_1912,VAR_1913,VAR_1914,VAR_1915,VAR_1916,VAR_1917,VAR_1918,VAR_1919,VAR_1920,VAR_1921,VAR_1922,VAR_1923,VAR_1924,VAR_1925,VAR_1926,VAR_1927,VAR_1928,VAR_1929,VAR_1930,VAR_1931,VAR_1932,VAR_1933,0001H,0001R,0001Q,0005C,0005B,0005N,0005S,1934_IAPS,1934_RCC,1934_BRANCH,1934_MOBILE,1934_CSC,target1
0,2,224,0,4300,5,0,0,5,3,0,0,0,84,49463,0,116783,3230,1,998,998,998,998,66,9996,96,96,9996,9996,9996,9996,9996,96,96,0,0,0,0,0,0,0,...,98.0,98.0,98.0,98.0,998.0,999999998.0,999999998.0,999999998.0,999999998.0,998.0,998.0,9998.0,9998.0,0.0,98.0,999999998.0,999999998.0,9998.0,0.0,98.0,98.0,998.0,999999998.0,998.0,998.0,9998.0,9998.0,1,0,0,1,1,1,1,1.0,0.0,0.0,0.0,0.0,0.0
1,4,7,53,4448,0,0,0,3,12,0,0,0,63,303472,0,346196,899,1,998,998,998,998,99,9996,96,96,9996,9996,9996,9996,9996,96,96,9,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,6546.0,6546.0,6125.0,76.0,1.0,1.0,9996.0,111.0,0.0,98.0,999999998.0,999999998.0,9998.0,0.0,98.0,98.0,998.0,999999998.0,998.0,998.0,9998.0,9998.0,1,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0
2,5,116,3,3464,0,0,0,1,2,0,0,0,48,94990,0,122601,0,1,998,998,998,998,87,9996,96,96,21,64,9996,9996,9996,96,96,25,0,0,0,0,0,0,...,0.0,97.0,0.0,97.0,997.0,999999997.0,999999997.0,999999997.0,999999997.0,1.0,997.0,9996.0,113.0,0.0,98.0,999999998.0,999999998.0,9998.0,0.0,98.0,98.0,998.0,999999998.0,998.0,998.0,9998.0,9998.0,1,0,0,1,1,1,1,1.0,0.0,0.0,0.0,0.0,0.0
3,7,240,300,3200,0,0,0,1,6,2,0,0,65,20593,0,59490,286,1,998,998,400,400,77,9996,1,1,3,78,75,74,71,16,16,19,0,0,0,0,0,0,...,98.0,98.0,98.0,98.0,998.0,999999998.0,999999998.0,999999998.0,999999998.0,998.0,998.0,9998.0,9998.0,0.0,98.0,999999998.0,999999998.0,9998.0,0.0,98.0,98.0,998.0,999999998.0,998.0,998.0,9998.0,9998.0,1,0,0,1,1,1,1,0.0,1.0,0.0,0.0,0.0,0.0
4,8,72,261,2000,0,0,0,1,8,0,0,0,84,10071,0,35708,947,1,1,30,998,998,57,9996,96,96,41,35,9996,9996,9996,96,96,0,0,0,0,0,0,0,...,98.0,98.0,98.0,98.0,998.0,999999998.0,999999998.0,999999998.0,999999998.0,998.0,998.0,9998.0,9998.0,0.0,98.0,999999998.0,999999998.0,9998.0,0.0,98.0,98.0,998.0,999999998.0,998.0,998.0,9998.0,9998.0,0,1,0,0,0,0,0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80249,160768,12,5,1660,0,0,1,0,4,2,0,0,54,1678,0,6000,500,1,998,998,998,998,45,9996,96,96,9996,9996,9996,9996,9996,96,96,0,0,0,0,0,0,0,...,98.0,98.0,98.0,98.0,998.0,999999998.0,999999998.0,999999998.0,999999998.0,998.0,998.0,9998.0,9998.0,0.0,98.0,999999998.0,999999998.0,9998.0,0.0,98.0,98.0,998.0,999999998.0,998.0,998.0,9998.0,9998.0,0,1,0,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0
80250,160773,2,0,2100,0,0,0,0,1,1,0,1,4,710,7227,1119,0,1,998,998,998,998,63,9996,1,1,4,62,56,9990,52,14,5,6,0,0,0,0,0,0,...,98.0,98.0,98.0,98.0,998.0,999999998.0,999999998.0,999999998.0,999999998.0,998.0,998.0,9998.0,9998.0,0.0,98.0,999999998.0,999999998.0,9998.0,0.0,98.0,98.0,998.0,999999998.0,998.0,998.0,9998.0,9998.0,0,1,0,1,1,1,1,1.0,0.0,0.0,0.0,0.0,0.0
80251,160774,92,142,1693,1,0,0,1,1,1,0,0,74,56115,0,74524,993,1,998,998,998,998,77,9996,1,1,2,79,73,71,9996,19,8,20,0,0,0,0,0,0,...,98.0,98.0,98.0,98.0,998.0,999999998.0,999999998.0,999999998.0,999999998.0,998.0,998.0,9998.0,9998.0,0.0,98.0,999999998.0,999999998.0,9998.0,0.0,98.0,98.0,998.0,999999998.0,998.0,998.0,9998.0,9998.0,1,0,0,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0
80252,160775,24,38,2282,0,0,0,0,1,2,0,0,48,298,0,500,200,1,1,120,998,998,

# Training using XGBoost Classifier

In [72]:
featureNames = df_train.columns[1:-1]


In [81]:
df_train = df_train.sample(n=20000)
#Training through XGBoost Classifier
dtrain = xgb.DMatrix(df_train[featureNames].values, label=df_train['target1'].values)

param = {'max_depth':2, 
         'eta':1, 
         'objective':'binary:logistic', 
         'eval_metric': 'auc'}
clf = xgb.train(param, dtrain, 20)

#Prediction in batches due to memory constraints
submission = df_test[['ID']]
submission['target1'] = np.nan
step = len(submission)/10000
#for rows in range(0, len(submission), int(step)):
    #submission.loc[rows:rows+step, "target"] = clf.predict(xgb.DMatrix(df_test.loc[rows:rows+step, featureNames].values))


submission.to_csv("xgboost_submission.csv", index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
